# Ticket To Ride Map Editor (Interactive)

Use the widgets to tweak routes, city locations, and colors. Changes update the in-notebook visualization and stay in memory until you export.


In [1]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import ipywidgets as widgets
from IPython.display import display

board = json.loads(Path("board.json").read_text())

places = {p["id"]: p for p in board.get("places", [])}

special_cities = {
    "Boston",
    "Philadelphia",
    "Washington DC",
    "Long Beach",
    "Riverside",
    "Fairbanks",
    "Tashkent",
    "Almaty",
    "Yerevan",
    "Daegu",
    "Goesan",
    "Seoul",
}
roads = board.get("roads", [])

dimensions = board.get("dimensions", {}).get("full", {})
width = dimensions.get("width", 72.44)
height = dimensions.get("height", 47.04)

bg_color = "#9fb3c6"

color_map = {
    "red": "#d72638",
    "blue": "#1f77b4",
    "green": "#2ca02c",
    "yellow": "#f2c94c",
    "black": "#111111",
    "white": "#f8f6f1",
    "orange": "#f2994a",
    "purple": "#8e44ad",
    "grey": "#7f8c8d",
}

continent_by_city = {
    "Fairbanks": "North America",
    "Berkeley": "North America",
    "Riverside": "North America",
    "Long Beach": "North America",
    "Hilo": "North America",
    "Fort Worth": "North America",
    "Columbus": "North America",
    "Boston": "North America",
    "Philadelphia": "North America",
    "Washington DC": "North America",
    "Seoul": "Asia",
    "Daegu": "Asia",
    "Goesan": "Asia",
    "Busan": "Asia",
    "Tokyo": "Asia",
    "Dili": "Asia",
    "Almaty": "Asia",
    "Tashkent": "Asia",
    "Mandalay": "Asia",
    "Yangon": "Asia",
    "Mawlamyine": "Asia",
    "Beijing": "Asia",
    "Shanghai": "Asia",
    "Istanbul": "Europe",
    "Izmir": "Europe",
    "Moscow": "Europe",
    "Tbilisi": "Europe",
    "Yerevan": "Europe",
}

def continent_for_city(name):
    return continent_by_city.get(name, "unknown")


def route_kind_for_road(road):
    override = road.get("routeType")
    if override in {"train", "ocean"}:
        return override
    ids = road.get("placeIds", [])
    if len(ids) != 2:
        return "train"
    name_a = places.get(ids[0], {}).get("name")
    name_b = places.get(ids[1], {}).get("name")
    cont_a = continent_for_city(name_a) if name_a else "unknown"
    cont_b = continent_for_city(name_b) if name_b else "unknown"
    if cont_a != "unknown" and cont_a == cont_b:
        return "train"
    return "ocean"



def bezier_control(p1, p2, curvature):
    p1 = np.array(p1, dtype=float)
    p2 = np.array(p2, dtype=float)
    vec = p2 - p1
    dist = np.hypot(vec[0], vec[1])
    if dist == 0:
        return p1, np.array([0.0, 0.0]), 0.0
    normal = np.array([-vec[1], vec[0]]) / dist
    offset = (curvature / 100.0) * dist
    ctrl = (p1 + p2) / 2.0 + normal * offset
    return ctrl, normal, dist


def bezier_points(p1, p2, ctrl, steps=200):
    p1 = np.array(p1, dtype=float)
    p2 = np.array(p2, dtype=float)
    t = np.linspace(0.0, 1.0, steps)
    pts = (1 - t)[:, None] ** 2 * p1 + 2 * (1 - t)[:, None] * t[:, None] * ctrl + t[:, None] ** 2 * p2
    return pts


def draw_segment(ax, a, b, curvature, lanes, space_amount, route_kind, is_tunnel, lane_sep=0.25):
    lanes = lanes or [{"colour": "grey"}]
    is_ocean = route_kind == "ocean"
    lane_count = max(1, len(lanes))
    base_ctrl, normal, _ = bezier_control(a, b, curvature)

    for idx, lane in enumerate(lanes):
        offset = (idx - (lane_count - 1) / 2) * lane_sep
        a_shift = (np.array(a) + normal * offset).tolist()
        b_shift = (np.array(b) + normal * offset).tolist()
        ctrl_shift = (np.array(base_ctrl) + normal * offset).tolist()
        pts = bezier_points(a_shift, b_shift, ctrl_shift)
        lane_color = color_map.get(lane.get("colour", "grey"), "#7f8c8d")
        line_style = "--" if is_ocean else "-"
        line_width = 2 if is_ocean else 3
        ax.plot(pts[:, 0], pts[:, 1], color=lane_color, linewidth=line_width, linestyle=line_style, solid_capstyle="round")

        if is_tunnel:
            tunnel_color = "#f8f6f1" if lane.get("colour") == "black" else "#111111"
            ax.plot(pts[:, 0], pts[:, 1], color=tunnel_color, linewidth=1.4, linestyle=(0, (2, 2)), alpha=0.9)

        if space_amount:
            t_marks = np.linspace(0.15, 0.85, space_amount)
            p1_arr = np.array(a_shift)
            p2_arr = np.array(b_shift)
            ctrl_arr = np.array(ctrl_shift)
            pts_marks = (1 - t_marks)[:, None] ** 2 * p1_arr +                 2 * (1 - t_marks)[:, None] * t_marks[:, None] * ctrl_arr +                 t_marks[:, None] ** 2 * p2_arr
            ax.scatter(
                pts_marks[:, 0],
                pts_marks[:, 1],
                s=20,
                c="none" if is_ocean else lane_color,
                edgecolors=lane_color if is_ocean else "black",
                linewidths=0.8 if is_ocean else 0.5,
                zorder=4,
            )


def draw_road(ax, road, lane_sep=0.25):
    place_ids = road.get("placeIds", [])
    if len(place_ids) != 2:
        return
    p1 = places.get(place_ids[0])
    p2 = places.get(place_ids[1])
    if not p1 or not p2:
        return

    a = (p1["coordinate"]["x"], p1["coordinate"]["y"])
    b = (p2["coordinate"]["x"], p2["coordinate"]["y"])
    curvature = -road.get("curvature", 0)
    lanes = road.get("lanes", [])
    space_amount = road.get("spaceAmount", 0)
    route_kind = route_kind_for_road(road)

    if road.get("wrapAround"):
        if a[0] <= b[0]:
            left_city, right_city = a, b
        else:
            left_city, right_city = b, a
        wrap_y = (left_city[1] + right_city[1]) / 2.0
        left_exit = (0.0, wrap_y)
        right_entry = (width, wrap_y)
        left_count = space_amount // 2
        right_count = space_amount - left_count
        draw_segment(ax, left_city, left_exit, curvature, lanes, left_count, route_kind, road.get("tunnelCostDouble"), lane_sep)
        draw_segment(ax, right_entry, right_city, curvature, lanes, right_count, route_kind, road.get("tunnelCostDouble"), lane_sep)
        return

    draw_segment(ax, a, b, curvature, lanes, space_amount, route_kind, road.get("tunnelCostDouble"), lane_sep)



def draw_board():
    fig, ax = plt.subplots(figsize=(12, 8))
    fig.patch.set_facecolor(bg_color)
    ax.set_facecolor(bg_color)

    frame = patches.Rectangle((0, 0), width, height, fill=False, edgecolor="#2c3e50", linewidth=1.5, zorder=2)
    ax.add_patch(frame)

    for road in roads:
        draw_road(ax, road)

    normal_xs = []
    normal_ys = []
    special_xs = []
    special_ys = []
    for p in places.values():
        name = p.get("name", "")
        x = p["coordinate"]["x"]
        y = p["coordinate"]["y"]
        if name in special_cities:
            special_xs.append(x)
            special_ys.append(y)
        else:
            normal_xs.append(x)
            normal_ys.append(y)
    ax.scatter(normal_xs, normal_ys, s=50, c="#2c3e50", edgecolors="white", linewidths=1.0, zorder=5)
    ax.scatter(special_xs, special_ys, s=90, c="#2c3e50", marker="s", edgecolors="white", linewidths=1.2, zorder=6)

    for p in places.values():
        x = p["coordinate"]["x"]
        y = p["coordinate"]["y"]
        label = p.get("label", {})
        offset = label.get("offset", {"x": 0.0, "y": 0.0})
        ax.text(
            x + offset.get("x", 0.0),
            y + offset.get("y", 0.0),
            p.get("name", ""),
            fontsize=8,
            color="#1f2d3d",
            ha="center",
            va="center",
            zorder=6,
        )

    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.set_aspect("equal")
    ax.set_title(board.get("info", {}).get("name", "board"))
    if show_bg_label:
        ax.text(0.6, height - 0.6, f"BG: {bg_color}", fontsize=8, color="#1f2d3d",
                ha="left", va="top", zorder=7,
                bbox=dict(boxstyle="round,pad=0.2", facecolor="#f2f2f2", edgecolor="#2c3e50", linewidth=0.8))

    ax.axis("off")
    plt.show()


def next_lane_id():
    return max((l.get("id", 0) for r in roads for l in r.get("lanes", [])), default=0) + 1


def find_road(place_a_id, place_b_id):
    key = {place_a_id, place_b_id}
    for road in roads:
        ids = road.get("placeIds", [])
        if len(ids) == 2 and {ids[0], ids[1]} == key:
            return road
    return None

def format_route_status(road):
    if not road:
        return "<b>Status:</b> Unbuilt"
    status = "Built"
    if road.get("wrapAround"):
        status = "Built (wraparound)"
    if road.get("routeType") in {"train", "ocean"}:
        status += f" | {road.get('routeType')}"
    lanes = road.get("lanes", [])
    if len(lanes) > 1:
        status += " | double"
    return f"<b>Status:</b> {status}"

def sync_main_route_controls():
    global syncing_controls
    if syncing_controls:
        return
    syncing_controls = True
    road = find_road(place_a.value, place_b.value)
    if road:
        space_amount.value = int(road.get("spaceAmount", space_amount.value) or space_amount.value)
        curvature.value = int(road.get("curvature", 0) or 0)
        lanes = road.get("lanes", [])
        if lanes:
            lane1_colour.value = lanes[0].get("colour", "grey")
            if len(lanes) > 1:
                route_double.value = True
                lane2_colour.value = lanes[1].get("colour", "grey")
            else:
                route_double.value = False
        else:
            lane1_colour.value = "grey"
            route_double.value = False
        override = road.get("routeType")
        route_type_override.value = override if override in {"train", "ocean"} else "auto"
        route_tunnel.value = bool(road.get("tunnelCostDouble"))
        route_status.value = format_route_status(road)
    else:
        route_status.value = format_route_status(None)
        lane1_colour.value = "grey"
        route_double.value = False
        route_type_override.value = "auto"
        route_tunnel.value = False
    syncing_controls = False



def sync_lr_route_controls():
    global syncing_controls
    if syncing_controls:
        return
    syncing_controls = True
    road = find_road(left_city.value, right_city.value)
    if road:
        lr_space_amount.value = int(road.get("spaceAmount", lr_space_amount.value) or lr_space_amount.value)
        lr_curvature.value = int(road.get("curvature", 0) or 0)
        lanes = road.get("lanes", [])
        if lanes:
            lr_colour.value = lanes[0].get("colour", "grey")
        else:
            lr_colour.value = "grey"
        lr_status.value = format_route_status(road)
    else:
        lr_colour.value = "grey"
        lr_status.value = format_route_status(None)
    syncing_controls = False



place_items = [(p["name"], p["id"]) for p in places.values()]
place_items = sorted(place_items, key=lambda x: x[0])

left_place_items = [item for item in place_items if places[item[1]]["coordinate"]["x"] < width / 2]
right_place_items = [item for item in place_items if places[item[1]]["coordinate"]["x"] >= width / 2]

place_a = widgets.Dropdown(description="City A", options=place_items)
place_b = widgets.Dropdown(description="City B", options=place_items)

space_amount = widgets.IntSlider(description="Trains", min=1, max=6, value=3)
curvature = widgets.IntSlider(description="Curvature", min=-80, max=80, value=0)
route_type_override = widgets.ToggleButtons(
    description="Type",
    options=[("Auto", "auto"), ("Train", "train"), ("Ocean", "ocean")],
    value="auto",
)
route_double = widgets.Checkbox(description="Double Route", value=False)
lane1_colour = widgets.Dropdown(description="Lane 1", options=list(color_map.keys()), value="grey")
lane2_colour = widgets.Dropdown(description="Lane 2", options=list(color_map.keys()), value="grey")
route_tunnel = widgets.Checkbox(description="Tunnel (x2 cost)", value=False)
route_status = widgets.HTML(value="<b>Status:</b> Unbuilt")

add_update = widgets.Button(description="Add/Update Route", button_style="primary")
remove_route = widgets.Button(description="Remove Route", button_style="danger")

city_select = widgets.Dropdown(description="City", options=place_items)
city_x = widgets.FloatSlider(description="X", min=0, max=width, step=0.1, value=0.0)
city_y = widgets.FloatSlider(description="Y", min=0, max=height, step=0.1, value=0.0)
city_update = widgets.Button(description="Update City Location", button_style="warning")

left_city = widgets.Dropdown(description="Left", options=left_place_items)
right_city = widgets.Dropdown(description="Right", options=right_place_items)
connect_lr = widgets.Button(description="Connect L->R", button_style="info")

refresh_plot = widgets.Button(description="Refresh Plot", button_style="")
toggle_bg_label = widgets.Button(description="Toggle BG Label", button_style="")

lr_space_amount = widgets.IntSlider(description="Trains", min=1, max=6, value=3)
lr_curvature = widgets.IntSlider(description="Curvature", min=-80, max=80, value=0)
lr_colour = widgets.Dropdown(description="Color", options=list(color_map.keys()), value="grey")
lr_status = widgets.HTML(value="<b>Status:</b> Unbuilt")

output = widgets.Output()
show_bg_label = False
syncing_controls = False


def sync_city_sliders(change=None):
    p = places.get(city_select.value)
    if not p:
        return
    city_x.value = p["coordinate"]["x"]
    city_y.value = p["coordinate"]["y"]


def update_route(_):
    a_id = place_a.value
    b_id = place_b.value
    if a_id == b_id:
        return
    road = find_road(a_id, b_id)
    if road is None:
        lanes = [{"id": next_lane_id(), "colour": lane1_colour.value}]
        if route_double.value:
            lanes.append({"id": next_lane_id(), "colour": lane2_colour.value})
        road = {
            "id": max((r.get("id", 0) for r in roads), default=0) + 1,
            "placeIds": [a_id, b_id],
            "spaceAmount": space_amount.value,
            "lanes": lanes,
            "curvature": curvature.value,
        }
        if route_type_override.value != "auto":
            road["routeType"] = route_type_override.value
        else:
            road.pop("routeType", None)
        if route_tunnel.value:
            road["tunnelCostDouble"] = True
        else:
            road.pop("tunnelCostDouble", None)
        if route_tunnel.value:
            road["tunnelCostDouble"] = True
        else:
            road.pop("tunnelCostDouble", None)
        roads.append(road)
    else:
        road["spaceAmount"] = space_amount.value
        road["curvature"] = curvature.value
        lane_id = road.get("lanes", [{}])[0].get("id", 0) or next_lane_id()
        lanes = [{"id": lane_id, "colour": lane1_colour.value}]
        if route_double.value:
            existing_lanes = road.get("lanes", [])
            if len(existing_lanes) > 1:
                lane2_id = existing_lanes[1].get("id", 0) or next_lane_id()
            else:
                lane2_id = next_lane_id()
            lanes.append({"id": lane2_id, "colour": lane2_colour.value})
        road["lanes"] = lanes
        if route_type_override.value != "auto":
            road["routeType"] = route_type_override.value
        else:
            road.pop("routeType", None)
        if route_tunnel.value:
            road["tunnelCostDouble"] = True
        else:
            road.pop("tunnelCostDouble", None)
        if route_tunnel.value:
            road["tunnelCostDouble"] = True
        else:
            road.pop("tunnelCostDouble", None)

    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()




def remove_route_handler(_):
    road = find_road(place_a.value, place_b.value)
    if road:
        roads.remove(road)
    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()

def update_city(_):
    p = places.get(city_select.value)
    if not p:
        return
    p["coordinate"]["x"] = city_x.value
    p["coordinate"]["y"] = city_y.value
    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()


def connect_left_right(_):
    if left_city.value == right_city.value:
        return
    a_id = left_city.value
    b_id = right_city.value
    road = find_road(a_id, b_id)
    if road is None:
        road = {
            "id": max((r.get("id", 0) for r in roads), default=0) + 1,
            "placeIds": [a_id, b_id],
            "spaceAmount": lr_space_amount.value,
            "lanes": [{"id": max((l.get("id", 0) for r in roads for l in r.get("lanes", [])), default=0) + 1, "colour": lr_colour.value}],
            "curvature": lr_curvature.value,
        }
        road["wrapAround"] = True
        roads.append(road)
    else:
        road["spaceAmount"] = lr_space_amount.value
        road["curvature"] = lr_curvature.value
        road["lanes"] = [{"id": road.get("lanes", [{}])[0].get("id", 0) or 1, "colour": lr_colour.value}]
        road["wrapAround"] = True
    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()


def refresh_plot_handler(_):
    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()

def toggle_bg_label_handler(_):
    global show_bg_label
    show_bg_label = not show_bg_label
    refresh_plot_handler(_)

def refresh_view():
    with output:
        output.clear_output(wait=True)
        draw_board()
    sync_lr_route_controls()
    sync_main_route_controls()

city_select.observe(sync_city_sliders, names="value")
place_a.observe(lambda change: sync_main_route_controls(), names="value")
place_b.observe(lambda change: sync_main_route_controls(), names="value")
left_city.observe(lambda change: sync_lr_route_controls(), names="value")
right_city.observe(lambda change: sync_lr_route_controls(), names="value")
add_update.on_click(update_route)
remove_route.on_click(remove_route_handler)
city_update.on_click(update_city)
connect_lr.on_click(connect_left_right)
refresh_plot.on_click(refresh_plot_handler)
toggle_bg_label.on_click(toggle_bg_label_handler)

sync_city_sliders()
sync_main_route_controls()
sync_lr_route_controls()
refresh_view()

controls_routes = widgets.VBox([
    widgets.HTML("<b>Routes</b>"),
    place_a,
    place_b,
    route_status,
    space_amount,
    curvature,
    route_type_override,
    route_double,
    route_tunnel,
    lane1_colour,
    lane2_colour,
    add_update,
    remove_route,
])

controls_city = widgets.VBox([
    widgets.HTML("<b>City Location</b>"),
    city_select,
    city_x,
    city_y,
    city_update,
])

controls_lr = widgets.VBox([
    widgets.HTML("<b>Connect Left to Right</b>"),
    left_city,
    right_city,
    lr_status,
    lr_space_amount,
    lr_curvature,
    lr_colour,
    connect_lr,
    refresh_plot,
    toggle_bg_label,
])

ui = widgets.HBox([controls_routes, controls_city, controls_lr])

display(ui, output)


Output()

In [2]:
# Save current in-memory edits back to board.json
save_button = widgets.Button(description="Save to board.json", button_style="success")


def save_board(_):
    Path("board.json").write_text(json.dumps(board, indent=2))
    with output:
        print("Saved to board.json")

save_button.on_click(save_board)

display(save_button)


Button(button_style='success', description='Save to board.json', style=ButtonStyle())